# Advanced Machine Learning Application - Assignment 2
### Rohan Rocky Britto - Student ID: 24610990

## Data Import and Preparation

Import required packages

In [1]:
import pandas as pd
import numpy as np

Reading csv files into dataframe

In [2]:
df_train = pd.read_csv('../../data/raw/sales_train.csv')
df_test = pd.read_csv('../../data/raw/sales_test.csv')
df_calendar = pd.read_csv('../../data/raw/calendar.csv')
df_events = pd.read_csv('../../data/raw/calendar_events.csv')
df_sell_prices = pd.read_csv('../../data/raw/items_weekly_sell_prices.csv')

Let us jave a quick look at the data

In [3]:
df_train.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1532,d_1533,d_1534,d_1535,d_1536,d_1537,d_1538,d_1539,d_1540,d_1541
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,1,0,1,0,1,0,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,8,2,0,8,2,3,1,1,3,8
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,0,1,3,2,1,1,2,2,3


In [4]:
df_test.head()

,d_1542,d_1543,d_1544,d_1545,d_1546,d_1547,d_1548,d_1549,d_1550,d_1551,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,0,1,0,2,1,0,2,0,1,0,...,2,4,0,0,0,0,3,3,0,1
1,0,0,0,0,0,0,0,0,1,0,...,0,1,2,1,1,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,4,1,0,1,3,5,2,3,0,2,...,1,1,0,4,0,1,3,0,2,6
4,3,0,0,1,1,0,2,0,2,1,...,0,0,0,2,1,0,0,2,1,0


In [5]:
df_calendar.head()

,date,wm_yr_wk,d
0,2011-01-29,11101,d_1
1,2011-01-30,11101,d_2
2,2011-01-31,11101,d_3
3,2011-02-01,11101,d_4
4,2011-02-02,11101,d_5


In [6]:
df_events.head()

,date,event_name,event_type
0,2011-02-06,SuperBowl,Sporting
1,2011-02-14,ValentinesDay,Cultural
2,2011-02-21,PresidentsDay,National
3,2011-03-09,LentStart,Religious
4,2011-03-16,LentWeek2,Religious


In [7]:
df_sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


Test file has only date fields. The other fields need to be fetched from train csv, hence, I have concatenated them

In [8]:
df_test = pd.concat([df_train[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']], df_test], axis=1)

The dates are stored in columns which leads to different columns in train and test files. We cannot train and test the model on different columns. We need to use melt function to unpivot the data and get the dates into rows.

In [9]:
df_train = df_train.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], var_name='d', value_name='items_sold')

Deleting rows where items_sold is 0 on a particular day and reseting the index of the dataframe

In [10]:
df_train = df_train[df_train['items_sold']!=0]
df_train.reset_index(drop=True, inplace=True)

Joining the events with the calendar based on date and setting 'None' when there are no events on a date

In [11]:
df_calendar = df_calendar.join(df_events.set_index('date'), on='date').fillna('None')

Joining the train and test dataframes with calendar and event details

In [12]:
df_train = df_train.join(df_calendar.set_index('d'), on='d')

Joining the train and test dataframes with sell prices based on store id, item id and week number

In [13]:
df_train = df_train.join(df_sell_prices.set_index(['store_id', 'item_id', 'wm_yr_wk']), on=['store_id', 'item_id', 'wm_yr_wk'])

In [14]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_train['num_date'] = df_train['date'].dt.strftime('%Y%m%d')
df_train['day_of_week'] = df_train['date'].dt.dayofweek

In [15]:
df_train.sample(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,event_name,event_type,sell_price,num_date,day_of_week
9452288,HOUSEHOLD_1_346_CA_3_evaluation,HOUSEHOLD_1_346,HOUSEHOLD_1,HOUSEHOLD,CA_3,CA,d_1128,1,2014-03-01,11405,None,None,3.98,20140301,5
780938,FOODS_3_116_WI_3_evaluation,FOODS_3_116,FOODS_3,FOODS,WI_3,WI,d_121,3,2011-05-29,11118,None,None,1.00,20110529,6
4445330,FOODS_3_057_TX_1_evaluation,FOODS_3_057,FOODS_3,FOODS,TX_1,TX,d_602,1,2012-09-21,11234,None,None,2.68,20120921,4
8850808,FOODS_2_079_CA_3_evaluation,FOODS_2_079,FOODS_2,FOODS,CA_3,CA,d_1068,2,2013-12-31,11349,None,None,3.87,20131231,1
8711086,FOODS_3_668_TX_2_evaluation,FOODS_3_668,FOODS_3,FOODS,TX_2,TX,d_1052,4,2013-12-15,11347,None,None,1.58,20131215,6


In [16]:
df_train['sale_revenue'] = df_train['items_sold'] * df_train['sell_price']

Having a quick look at the structure of the data after the transformations

In [17]:
df_train.shape

(14023073, 16)

In [18]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14023073 entries, 0 to 13984704
Data columns (total 16 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       object        
 2   dept_id       object        
 3   cat_id        object        
 4   store_id      object        
 5   state_id      object        
 6   d             object        
 7   items_sold    int64         
 8   date          datetime64[ns]
 9   wm_yr_wk      int64         
 10  event_name    object        
 11  event_type    object        
 12  sell_price    float64       
 13  num_date      object        
 14  day_of_week   int32         
 15  sale_revenue  float64       
dtypes: datetime64[ns](1), float64(2), int32(1), int64(2), object(10)
memory usage: 1.7+ GB


In [19]:
df_train.describe(include='all')

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,event_name,event_type,sell_price,num_date,day_of_week,sale_revenue
count,14023073,14023073,14023073,14023073,14023073,14023073,14023073,1.402307e+07,14023073,1.402307e+07,14023073,14023073,1.402307e+07,14023073,1.402307e+07,1.402307e+07
unique,30049,3049,7,3,10,3,1541,NaN,NaN,NaN,31,5,NaN,1541,NaN,NaN
top,FOODS_3_586_CA_2_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_3,CA,d_1234,NaN,NaN,NaN,None,None,NaN,20140615,NaN,NaN
freq,1543,15373,4977961,7917388,1792739,5951758,22290,NaN,NaN,NaN,12897112,12897112,NaN,22290,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.651708e+00,2013-05-24 12:28:54.247777792,1.130938e+04,NaN,NaN,3.784045e+00,NaN,3.110836e+00,1.018551e+01
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,2011-01-29 00:00:00,1.110100e+04,NaN,NaN,1.000000e-02,NaN,0.000000e+00,1.000000e-02
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,2012-06-03 00:00:00,1.121900e+04,NaN,NaN,1.970000e+00,NaN,1.000000e+00,3.470000e+00
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000e+00,2013-06-27 00:00:00,1.132200e+04,NaN,NaN,2.970000e+00,NaN,3.000000e+00,5.970000e+00
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000e+00,2014-06-07 00:00:00,1.141900e+04,NaN,NaN,4.880000e+00,NaN,5.000000e+00,1.184000e+01
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.630000e+02,2015-04-18 00:00:00,1.151200e+04,NaN,NaN,1.073200e+02,NaN,6.000000e+00,2.164320e+03


## Training and Validation split

In [20]:
df_train['date'].min()

Timestamp('2011-01-29 00:00:00')

In [21]:
df_train['date'].max()

Timestamp('2015-04-18 00:00:00')

We have approximately 4 years and 3 months worth of data. We will use the last 1 year data for validation as we have a huge dataset.

In [22]:
condition = df_train['date']>='01-04-2014'

In [23]:
df_validation = df_train[condition]

In [24]:
df_train = df_train[~condition]

In [25]:
df_train.to_csv('../../data/processed/train_processed.csv', index=False)

In [26]:
df_validation.to_csv('../../data/processed/validation_processed.csv', index=False)

## Data Preprocessing

I have not used some of the features for training the model. The features and the reasons are listed as follows:
  - id: It is an identifier. Using this will cause the model to overfit.
  - d, date: Year, month and day values have been extracted from the dates and hence, these columns are no longer required
  - event_name: This feature is not used for 2 reasons - firstly, it can create a bias, and secondly, there are a lot more events than the ones listed. The model may not perform well for a new event that gets added in the future

In [27]:
cat_cols = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_type']
num_cols = ['wm_yr_wk', 'sell_price', 'num_date', 'day_of_week']

Let us check the number of unique values in the categorical columns, so that we can decide the appropriate encoding method to be used

In [28]:
for cat_col in cat_cols:
    print('The number of unique values in column ', cat_col, ' is: ', len(df_train[cat_col].unique()))

The number of unique values in column  item_id  is:  2689
The number of unique values in column  dept_id  is:  7
The number of unique values in column  cat_id  is:  3
The number of unique values in column  store_id  is:  10
The number of unique values in column  state_id  is:  3
The number of unique values in column  event_type  is:  5


As there are a lot of unique values in item_id, dept_id and store_id fields, one-hot encoding them can lead to a lot of features in the dataset. I will be using target encoding on these columns and one-hot encoding on the other categorical features.

In [29]:
cat_ohe_cols = ['cat_id', 'state_id', 'event_type']
cat_tar_cols = ['item_id', 'dept_id', 'store_id']

Importing processing and pipeline related packages

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder
from joblib import dump, load

Creating a pipeline for standard scaling of numerical features, one-hot encoding of categorical features with few unique values and target encoding of categorical features with many unique values

In [31]:
num_transformer = Pipeline(
    steps=[
        ('scaler', StandardScaler())
    ]
)

In [32]:
cat_ohe_transformer = Pipeline(
    steps=[
        ('one_hot_encoder', OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore'))
    ]
)

In [33]:
cat_tar_transformer = Pipeline(
    steps=[
        ('target_encoder', TargetEncoder(random_state=8))
    ]
)

Creating a column transformer to build a column-wise transformation list

In [34]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num_cols', num_transformer, num_cols),
        ('cat_ohe_transformer', cat_ohe_transformer, cat_ohe_cols),
        ('cat_tar_transformer', cat_tar_transformer, cat_tar_cols)
    ]
)

Creating a preprocessor pipeline and storing it for future use

In [35]:
preprocessor_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor)
    ]
)

In [36]:
dump(preprocessor_pipe, '../../src/preprocessor_pipeline.joblib')

['../../src/preprocessor_pipeline.joblib']

In [37]:
train_target = df_train['sale_revenue']
validation_target = df_validation['sale_revenue']

## Baseline Model

Let us build a baseline model with mean values and test it using MAE and RMSE scores

In [38]:
mean_value = train_target.mean()
base_preds = np.full((len(train_target), 1), mean_value)

In [39]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [40]:
print('The Mean Absolute Error for the baseline model is ', mean_absolute_error(train_target, base_preds))
print('The Root Mean Squared Error for training set is ', mean_squared_error(train_target, base_preds, squared=False))

The Mean Absolute Error for the baseline model is  7.825201167624908
The Root Mean Squared Error for training set is  14.701932097001018


## Model Building

Let us start by building a Linear Regression model and check if it is able to perform well

In [41]:
from sklearn.linear_model import LinearRegression

In [42]:
lin_pipe = Pipeline(
    steps=[
        ('preprocessor', preprocessor_pipe),
        ('lin', LinearRegression())
    ]
)

In [43]:
lin_pipe.fit(df_train, train_target)

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('preprocessor',
                                  ColumnTransformer(transformers=[('num_cols',
                                                                   Pipeline(steps=[('scaler',
                                                                                    StandardScaler())]),
                                                                   ['wm_yr_wk',
                                                                    'sell_price',
                                                                    'num_date',
                                                                    'day_of_week']),
                                                                  ('cat_ohe_transformer',
                                                                   Pipeline(steps=[('one_hot_encoder',
                                                                                    OneHotEncoder(drop='first',
                                                                                                  handle_unknown='ignore',
                                                                                                  sparse_output=False))]),
                                                                   ['cat_id',
                                                                    'state_id',
                                                                    'event_type']),
                                                                  ('cat_tar_transformer',
                                                                   Pipeline(steps=[('target_encoder',
                                                                                    TargetEncoder(random_state=8))]),
                                                                   ['item_id',
                                                                    'dept_id',
                                                                    'store_id'])]))])),
                ('lin', LinearRegression())])

In [44]:
train_preds = lin_pipe.predict(df_train)

In [45]:
validation_preds = lin_pipe.predict(df_validation)

Store the Linear Regression pipeline for future use

In [46]:
dump(lin_pipe, '../../models/predictive/lin_pipe.joblib')

['../../models/predictive/lin_pipe.joblib']

## Evaluation

Defining a function to evaluate the performance of the models on training and validation datasets. We will store this function in a python file for future use

In [47]:
def evaluate_model(train_target, train_preds, validation_target, validation_preds):
    
    print('The Mean Absolute Error for training set is ', mean_absolute_error(train_target, train_preds))
    print('The Mean Absolute Error for validation set is ', mean_absolute_error(validation_target, validation_preds))
    
    print('The Root Mean Squared Error for training set is ', mean_squared_error(train_target, train_preds, squared=False))
    print('The Root Mean Squared Error for validation set is ', mean_squared_error(validation_target, validation_preds, squared=False))

In [48]:
evaluate_model(train_target, train_preds, validation_target, validation_preds)

The Mean Absolute Error for training set is  5.628608338755452
The Mean Absolute Error for validation set is  5.546183039844255
The Root Mean Squared Error for training set is  11.037705840930432
The Root Mean Squared Error for validation set is  11.118623683380077


Looking at the MAE and MSE scores, the model seems to be performing slightly better than the base model. Let us compare the prediction values with the actual values by adding it in the dataframe and checking some sample cases.

In [49]:
df_train['prediction'] = train_preds

In [50]:
df_validation['prediction'] = validation_preds

In [51]:
df_train.sample(5)

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,event_name,event_type,sell_price,num_date,day_of_week,sale_revenue,prediction
6838756,FOODS_2_139_TX_2_evaluation,FOODS_2_139,FOODS_2,FOODS,TX_2,TX,d_865,3,2013-06-11,11320,None,None,2.98,20130611,1,8.94,8.039880
2075541,HOBBIES_1_074_CA_3_evaluation,HOBBIES_1_074,HOBBIES_1,HOBBIES,CA_3,CA,d_309,4,2011-12-03,11145,None,None,3.67,20111203,5,14.68,13.690158
7235180,FOODS_3_090_CA_4_evaluation,FOODS_3_090,FOODS_3,FOODS,CA_4,CA,d_905,45,2013-07-21,11326,None,None,1.38,20130721,6,62.10,102.948337
2829131,HOBBIES_1_415_TX_1_evaluation,HOBBIES_1_415,HOBBIES_1,HOBBIES,TX_1,TX,d_409,2,2012-03-12,11207,None,None,6.68,20120312,0,13.36,8.892590
6813825,HOBBIES_1_149_CA_2_evaluation,HOBBIES_1_149,HOBBIES_1,HOBBIES,CA_2,CA,d_863,3,2013-06-09,11320,None,None,1.56,20130609,6,4.68,3.042233


**Conclusion:** The model is performing better than the base model. We will build and evaluate some other algorithms in the next experiments to check if we are able to achieve better scores.